In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from VarianceReduction import *

In [20]:
evaluate = VarianceReduction()

In [ ]:
df_ml = ml_train(evaluate.df)

In [ ]:
dim_abate_list, cuped_abate_list, cupac_abate_list, mlrate_abate_list, state_abate_list = eval_ate(df_ml,100,20000)

In [25]:
## variance 
DIM_VAR = Variance(dim_abate_list)
CUPED_VAR = Variance(cuped_abate_list)
CUPAC_VAR = Variance(cupac_abate_list)
MLRATE_VAR = Variance(mlrate_abate_list)
STATE_VAR = Variance(state_abate_list)

In [26]:
DIM_VAR, CUPED_VAR, CUPAC_VAR, MLRATE_VAR, STATE_VAR

(0.6392037246222972,
 0.3376782366920461,
 0.30556226669584835,
 0.30443661306100844,
 0.12344357573827186)

In [27]:
# Empirical coverage
DIM_EMP_COV = Emp_Cov(dim_abate_list, evaluate.ate)
CUPED_EMP_COV = Emp_Cov(cuped_abate_list, evaluate.ate)
CUPAC_EMP_COV = Emp_Cov(cupac_abate_list, evaluate.ate)
MLRATE_EMP_COV = Emp_Cov(mlrate_abate_list, evaluate.ate)
STATE_EMP_COV = Emp_Cov(state_abate_list, evaluate.ate)


In [ ]:
DIM_EMP_COV, CUPED_EMP_COV, CUPAC_EMP_COV, MLRATE_EMP_COV, STATE_EMP_COV